# hima image se angle during runtime

In [ ]:
import pandas as pd
import mediapipe as mp
import numpy as np
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

exr_df = pd.read_csv("EPC EXERCISE Sheet - Sheet1.csv")
exr_df['EXERCISE NAME'] = exr_df['EXERCISE NAME'].str.lower().str.replace('_', '')
exr_df['EXERCISE NAME'] = exr_df['EXERCISE NAME'].str.replace(' ', '')
exr_df.rename(columns={'FRONT /  SIDE VIEW':'View','VERTICAL / HORIZONTAL':'Orientation'},inplace=True)
exr_col = exr_df.columns
exr_df.columns = [i.upper() for i in exr_col]
exr_df.head(5)

yoga_df = pd.read_csv("EPC YOGA sheet - Sheet1.csv")
yoga_df['YOGA NAME'] = yoga_df['YOGA NAME'].str.lower().str.replace('_', '')
yoga_df['YOGA NAME'] = yoga_df['YOGA NAME'].str.replace(' ', '')
yoga_df.rename(columns={'FRONT /  SIDE VIEW':'View','VERTICAL / HORIZONTAL':'Orientation'},inplace=True)
yoga_col = yoga_df.columns
yoga_df.columns = [i.upper() for i in yoga_col]
yoga_df.head(5)


angle_dct_info = { 'LEFT-ANKLE': ['LEFT_FOOT_INDEX','LEFT_ANKLE','LEFT_KNEE'],
              'RIGHT-ANKLE': ['RIGHT_FOOT_INDEX','RIGHT_ANKLE','RIGHT_KNEE'],
              'LEFT-KNEE': ['LEFT_HIP','LEFT_KNEE','LEFT_ANKLE'],
              'RIGHT-KNEE': ['RIGHT_HIP','RIGHT_KNEE','RIGHT_ANKLE'],
              'HIP': ['LEFT_KNEE','LEFT_HIP','LEFT_SHOULDER'],
              'LEFT-TORSO':	['LEFT_SHOULDER','LEFT_HIP','LEFT_PERPENDICULAR_POINT'],
              'RIGHT-TORSO':	['RIGHT_SHOULDER','RIGHT_HIP','RIGHT_PERPENDICULAR_POINT'],
              'LEFT-SHOULDER': ['LEFT_HIP','LEFT_SHOULDER','LEFT_ELBOW'],
              'RIGHT-SHOULDER': ['RIGHT_HIP','RIGHT_SHOULDER','RIGHT_ELBOW'],
              'LEFT-ELBOW': ['LEFT_WRIST','LEFT_ELBOW','LEFT_SHOULDER'],
              'RIGHT-ELBOW': ['RIGHT_WRIST','RIGHT_ELBOW','RIGHT_SHOULDER'],		
              'LEFT-WRIST':	['LEFT_INDEX','LEFT_WRIST','LEFT_ELBOW'],
              'RIGHT-WRIST': ['RIGHT_INDEX','RIGHT_WRIST','RIGHT_ELBOW']
            }
#  'NECK':  ['','',''],


In [3]:
def calculate_angle(A, B, C):
    # Convert points to numpy arrays
    A = np.array(A)
    B = np.array(B)
    C = np.array(C)
   # Calculate the distances between points
    AB = np.linalg.norm(B - A)
    BC = np.linalg.norm(C - B)
    AC = np.linalg.norm(C - A)
    
    # Use the law of cosines to calculate the angle at point B
    radians = np.arccos((AB**2 + BC**2 - AC**2) / (2 * AB * BC))
    angle_degrees = np.degrees(radians)
    return angle_degrees


def get_imp_keypnt(Category, Pose, final_jnt_lst):
    dct = {}
    final_dct = {}
    if Category == 'Exercise':
        if Pose in exr_df['EXERCISE NAME'].values:
            # Filter the DataFrame based on the target_pose
            result_df = exr_df[exr_df['EXERCISE NAME'] == Pose]
            for jnt in list(exr_df.columns)[5:]:
                dct[jnt] = result_df[jnt].values[0]
    elif Category == 'Yoga':
        if Pose in yoga_df['YOGA NAME'].values:
            result_df = yoga_df[yoga_df['YOGA NAME'] == Pose]
            for jnt in list(yoga_df.columns)[5:]:
                dct[jnt] = result_df[jnt].values[0]
    else:
        print(f"Pose '{Pose}' not found in the DataFrame.")
        return None
    main_key_lst = list(dct.keys())
    for nm in main_key_lst:
        for i in final_jnt_lst:
            if nm in i:
                final_dct[i] = dct[nm]
    return final_dct            
        

#  keypoints

In [4]:
df_column_nm = []
landmark = mp_pose.PoseLandmark
landmark = list(landmark)
for i in landmark[9:]:
    name = mp_pose.PoseLandmark(i).name
    df_column_nm.append(name)
df_column_nm

['MOUTH_LEFT',
 'MOUTH_RIGHT',
 'LEFT_SHOULDER',
 'RIGHT_SHOULDER',
 'LEFT_ELBOW',
 'RIGHT_ELBOW',
 'LEFT_WRIST',
 'RIGHT_WRIST',
 'LEFT_PINKY',
 'RIGHT_PINKY',
 'LEFT_INDEX',
 'RIGHT_INDEX',
 'LEFT_THUMB',
 'RIGHT_THUMB',
 'LEFT_HIP',
 'RIGHT_HIP',
 'LEFT_KNEE',
 'RIGHT_KNEE',
 'LEFT_ANKLE',
 'RIGHT_ANKLE',
 'LEFT_HEEL',
 'RIGHT_HEEL',
 'LEFT_FOOT_INDEX',
 'RIGHT_FOOT_INDEX']

In [15]:
# for image
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
img_pth = '13.jpg'
img = cv2.imread(img_pth)
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
result = pose.process(img_rgb)
Category = 'Exercise'
Pose = 'Squats'
# adjusting larger images 
ht = img.shape[0]
wdt = img.shape[1]
if ht > wdt:
    ratio = ht / wdt
    desired_width = 400
    desired_height = int(ratio*desired_width)
else:
    ratio = wdt / ht
    desired_height = 400
    desired_width = int(ratio*desired_height)

# Resize the image
img = cv2.resize(img, (desired_width, desired_height))

dct_nm = {}
if result.pose_landmarks:
    landmark = result.pose_landmarks.landmark
    # print(mp_pose.PoseLandmark)
    for i,val in enumerate(landmark):
        name = mp_pose.PoseLandmark(i).name
        dct_nm[name] = val
# for the image
dct_nm['LEFT_PERPENDICULAR_POINT'] = {'x': result.pose_landmarks.landmark[23].x,
                                 'y': result.pose_landmarks.landmark[11].y}
dct_nm['RIGHT_PERPENDICULAR_POINT'] = {'x': result.pose_landmarks.landmark[24].x,
                                 'y': result.pose_landmarks.landmark[12].y}
extract_pnt = []

final_jnt_lst = list(angle_dct_info.keys()) 
final_dct = get_imp_keypnt(Category,Pose.lower(),final_jnt_lst)
print(final_dct)
# mapping main keys 
for keypnt in final_dct:
    if final_dct[keypnt] == 1:
        lst = angle_dct_info[keypnt]
        extract_pnt.append(lst[1])
        if lst[0] not in extract_pnt:
            extract_pnt.append(lst[0])
        if lst[2] not in extract_pnt:
            extract_pnt.append(lst[2])
extract_pnt

{'LEFT-KNEE': 1, 'RIGHT-KNEE': 1, 'HIP': 1, 'LEFT-TORSO': 1, 'RIGHT-TORSO': 1, 'LEFT-SHOULDER': 0, 'RIGHT-SHOULDER': 0, 'LEFT-ELBOW': 0, 'RIGHT-ELBOW': 0, 'LEFT-ANKLE': 0, 'RIGHT-ANKLE': 0, 'LEFT-WRIST': 0, 'RIGHT-WRIST': 0}


['LEFT_KNEE',
 'LEFT_HIP',
 'LEFT_ANKLE',
 'RIGHT_KNEE',
 'RIGHT_HIP',
 'RIGHT_ANKLE',
 'LEFT_HIP',
 'LEFT_SHOULDER',
 'LEFT_HIP',
 'LEFT_PERPENDICULAR_POINT',
 'RIGHT_HIP',
 'RIGHT_SHOULDER',
 'RIGHT_PERPENDICULAR_POINT']

In [4]:
# for image
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
img_pth = '13.jpg'
img = cv2.imread(img_pth)
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
result = pose.process(img_rgb)
Category = 'Exercise'
Pose = 'Squats'
# adjusting larger images 
ht = img.shape[0]
wdt = img.shape[1]
if ht > wdt:
    ratio = ht / wdt
    desired_width = 400
    desired_height = int(ratio*desired_width)
else:
    ratio = wdt / ht
    desired_height = 400
    desired_width = int(ratio*desired_height)

# Resize the image
img = cv2.resize(img, (desired_width, desired_height))

dct_nm = {}
if result.pose_landmarks:
    landmark = result.pose_landmarks.landmark
    # print(mp_pose.PoseLandmark)
    for i,val in enumerate(landmark):
        name = mp_pose.PoseLandmark(i).name
        dct_nm[name] = val

# for the image
dct_nm['LEFT_PERPENDICULAR_POINT'] = {'x': result.pose_landmarks.landmark[23].x,
                                 'y': result.pose_landmarks.landmark[11].y}
dct_nm['RIGHT_PERPENDICULAR_POINT'] = {'x': result.pose_landmarks.landmark[24].x,
                                 'y': result.pose_landmarks.landmark[12].y}
angle = {}

final_jnt_lst = list(angle_dct_info.keys()) 
final_dct = get_imp_keypnt(Category,Pose.lower(),final_jnt_lst)
print(final_dct)
# mapping main keys 
for joint in angle_dct_info:
    if final_dct[joint] == 1:
        a = angle_dct_info[joint][0]
        b = angle_dct_info[joint][1]
        c = angle_dct_info[joint][2]
        a_coordinate = [dct_nm[a].x,dct_nm[a].y]
        b_coordinate = [dct_nm[b].x,dct_nm[b].y]
        if c=='LEFT_PERPENDICULAR_POINT':
            c_coordinate = [dct_nm['LEFT_PERPENDICULAR_POINT']['x'],dct_nm['LEFT_PERPENDICULAR_POINT']['y']]
        elif c=='RIGHT_PERPENDICULAR_POINT':
            c_coordinate = [dct_nm['RIGHT_PERPENDICULAR_POINT']['x'],dct_nm['RIGHT_PERPENDICULAR_POINT']['y']]
        else:
            c_coordinate = [dct_nm[c].x,dct_nm[c].y]
        angl_val = calculate_angle(a_coordinate,b_coordinate,c_coordinate)
        angle[joint] = angl_val
    else:
        angle[joint] = -1
angle

{'LEFT-KNEE': 1, 'RIGHT-KNEE': 1, 'HIP': 1, 'LEFT-TORSO': 1, 'RIGHT-TORSO': 1, 'LEFT-SHOULDER': 0, 'RIGHT-SHOULDER': 0, 'LEFT-ELBOW': 0, 'RIGHT-ELBOW': 0, 'LEFT-ANKLE': 0, 'RIGHT-ANKLE': 0, 'LEFT-WRIST': 0, 'RIGHT-WRIST': 0}


{'LEFT-ANKLE': -1,
 'RIGHT-ANKLE': -1,
 'LEFT-KNEE': 95.07024112398845,
 'RIGHT-KNEE': 69.27865780549305,
 'HIP': 100.06979920883238,
 'LEFT-TORSO': 23.730311857705473,
 'RIGHT-TORSO': 36.31680571846834,
 'LEFT-SHOULDER': -1,
 'RIGHT-SHOULDER': -1,
 'LEFT-ELBOW': -1,
 'RIGHT-ELBOW': -1,
 'LEFT-WRIST': -1,
 'RIGHT-WRIST': -1}

In [5]:
# Define the box dimensions based on the user's height
user_height = int(input("ENTER YOUR HEIGHT IN CM :"))  # replace with actual user's height in pixels
align = input("ENTER YOUR EXERCISE ALIGNMENT ('V':Vertical  'H':Horizontal)")
print('USERS Height : ',user_height)


USERS Height :  155


In [6]:
cap = cv2.VideoCapture(0)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print('Null.Frame')
            break
        
        # height, width of frame
        frame_height, frame_width = frame.shape[:2]

        # calculating rectangle height and width  according to the frame
        x_center = frame_width // 2
        y_center = frame_height // 2
        rect_height = int(frame_height - (frame_height * 0.30)) # 30% of the frame
        rect_width = int(frame_width - (frame_width * 0.30)) # 30% of the frame
        
        # adjusting the height, width according to user's height and alignment
        if align == 'V':
            rect_height = int(frame_height - (frame_height * 0.30) + (user_height * 0.70)) # frame_height - 30% of the frame + 70% of user_height
        elif align == 'H':
            rect_width = int(frame_width - (frame_width * 0.30) + (user_height * 0.70)) # frame width - 30% of the frame _ 70% of user_height

        # calculating top-left and bottom-right corner
        top_left_x = x_center - rect_width // 2
        top_left_y = y_center - rect_height // 2
        bottom_right_x = top_left_x + rect_width
        bottom_right_y = top_left_y + rect_height

        # drawing rectangle
        cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), thickness=2)

        """detecting whether there is a person in box or not"""
        # cropped frame
        cropped_frame = frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
        
        # Make detection on cropped frame
        results = pose.process(frame)
        
        # checking whether there is a person or not
        font_face = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color = (255, 0, 0)
        thickness = 2

        if not results.pose_landmarks: # there is no person inside
            text = 'Get your fucking ass inside the fucking box'
            cv2.putText(frame, text,(top_left_x + 5, top_left_y), font_face, font_scale, color, thickness)
        else:
            text = 'Done...'
            cv2.putText(frame, text,(top_left_x + 5, top_left_y), font_face, font_scale, color, thickness)

            # # list of keypoints detected inside the box
            # detected_keypoints = []
            # for i, value in enumerate(results.pose_landmarks.landmark):
            #     detected_keypoints.append(mp_pose.PoseLandmark(i).name)

            # if required_keypoints not in detected_keypoints: # checking if person is inside properly or not
            #     text = 'Adjust position'
            #     cv2.putText(frame, text,(top_left_x + 5, top_left_y), font_face, font_scale, color, thickness)
            # else:
            #     text = 'Perfect Buddy......'
            #     cv2.putText(frame, text,(top_left_x + 5, top_left_y), font_face, font_scale, color, thickness)
            #     # Make detection on whole frame
            #     # results = pose.process(frame)
            

        cv2.imshow('Mediapipe Feed', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()